In [ ]:
%matplotlib inline

In [ ]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import numpy as np
from scipy.stats import linregress
import scipy.stats as st

In [ ]:
# Constants


In [ ]:
# Import Music and Unemployment CSV
music_unemployment_path = "../data/music_and_unemployment.csv"
music_unemployment = pd.read_csv(music_unemployment_path)
music_unemployment.head()

In [ ]:
# Aaron's Code

In [ ]:
# End of Aaron's Code

In [ ]:
# Albin's Code

In [ ]:
# End of Albin's Code

In [ ]:
# Kelsey's code

In [ ]:
# Group by the song's date
music_unemployment_gb = music_unemployment.groupby(["Year", "Month", "Day"])

# Find the average of unemployment rate and weighed valence for each date
rate_v_valence = music_unemployment_gb.mean()[["Unemployment Rate", "valence"]]

# Create a Scatter Graph
rate_v_valence.plot(kind="scatter", x = "Unemployment Rate", y = "valence")

# Calculate the correlation coefficient and linear regression model 
x_values = rate_v_valence["Unemployment Rate"]
y_values = rate_v_valence["valence"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,.45),fontsize=15,color="red")

plt.title("Unemployment Rate vs. Valence (Happiness)")
plt.xlabel("Unemployment Rate")
plt.ylabel("Valence (Happiness)")
plt.show()

## OR Does weighed valence add to or change the analysis

In [ ]:
# Create a new data point "Weighted Valence"
music_unemployment["weighed valence"] = music_unemployment["valence"] * (101 - music_unemployment["Placement"])
music_unemployment.head()

In [ ]:
# Group by the song's date
music_unemployment_gb = music_unemployment.groupby(["Year", "Month", "Day"])

# Find the average of unemployment rate and weighed valence for each date
rate_v_valence = music_unemployment_gb.mean()[["Unemployment Rate", "weighed valence"]]

# Create a Scatter Graph
rate_v_valence.plot(kind="scatter", x = "Unemployment Rate", y = "weighed valence")

# Calculate the correlation coefficient and linear regression model 
x_values = rate_v_valence["Unemployment Rate"]
y_values = rate_v_valence["weighed valence"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(5,22),fontsize=15,color="red")

plt.title("Unemployment Rate vs. Valence (Happiness)")
plt.xlabel("Unemployment Rate")
plt.ylabel("Weighed Valence (Happiness)")
plt.show()

In [ ]:
# End of Kelsey's Code

In [ ]:
# Will's Code

In [ ]:
# End of Will's Code